In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
class TextLSTM(nn.Module):
    def __init__(self, n_class, n_hidden, dtype):
        super(TextLSTM, self).__init__()
        self.n_class = n_class
        self.lstm = nn.LSTM(input_size = n_class, hidden_size = n_hidden)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
        
    def forward(self, X):
        input = X.transpose(0,1)
        hidden_state = Variable(torch.zeros(1, len(X), n_hidden))
        cell_state = Variable(torch.zeros(1, len(X), n_hidden))
        outputs, (_,_) = self.lstm(input, (hidden_state, cell_state)) 
        outputs = outputs[-1]
        model = torch.mm(outputs, self.W) + self.b
        return model
    

In [5]:
def data():
    context = []
    target = []
    context.append([['A', 'close-up', 'of', 'a', 'brown', 'horse.'],['A', 'horse', 'facing', 'the', 'camera', 'with', 'a', 'horse', 'in', 'the.background'], 
                    ['A', 'horse', 'standing', 'near', 'a', 'gate', 'with', 'another','horse', 'in', 'the', 'background.'], 
                    ['A', 'reddish', 'brown', 'horse', 'looking', 'over', 'a', 'fence.']])
    target.append([['A', 'young', 'horse', 'looks', 'into', 'the', 'camera', 'from', 'behind', 'a', 'fence.']])
    context.append([['A', 'black', 'and', 'white', 'photo', 'of', 'three', 'horses,', 'their', 'handlers,', 'and', 'three', 'onlookers.'], 
                    ['Black', 'and', 'white', 'scene', 'of', 'three', 'people', 'looking', 'at', 'three', 'horses.'], 
                    ['People', 'stand', 'in', 'the', 'arena', 'with', 'horses.'], 
                    ['The', 'horses', 'prepare', 'for', 'the', 'show.']])
    target.append([['Three', 'horses', 'are', 'shown', 'using', 'halters.']])
    context.append([['A', 'horse', 'galloping', 'while', 'wearing', 'the', 'number', '4.'], 
                    ['A', 'race', 'horse', 'galloping', 'without', 'a', 'rider.'], 
                    ['Brown', 'race', 'horse', 'wearing', 'race', 'mask', 'and', 'halter.'], 
                    ['Horse', 'in', 'a', 'competition.']])
    target.append([['The', 'racing', 'horse', 'is', 'number', 'four.']])
    context.append([['A', 'close', 'up', 'of', 'a', 'horse', 'wearing', 'a', 'blue', 'halter.'], 
                    ['An', 'upclose', 'shot', 'of', 'a', 'horse', 'in', 'the', 'barn.'], 
                    ['An', 'up', 'close', 'view', 'of', 'a', 'horse.'], 
                    ['Close', 'up', 'of', 'a', 'brown', 'horse', 'with', 'a', 'blue', 'halter.']])
    target.append([['Up', 'close', 'view', 'of', 'horse', 'with', 'halter', 'attached', 'to', 'face.']])
    context.append([['a', 'bald', 'man', 'smiling', 'with', 'a', 'black,', 'white-haired', 'horse'], 
                    ['A', 'bald', 'man', 'touches', 'the', 'nose', 'of', 'a', 'black', 'horse', 'on', 'the', 'other', 'side', 'of', 'a', 'fence.'], 
                    ['A', 'bald', 'man', 'with', 'a', 'beard', 'pets', 'the', 'nose', 'of', 'a', 'black', 'and', 'white', 'horse', 'at', 'the', 'fence.'], 
                    ['A', 'man', 'is', 'staring', 'at', 'a', 'horse', 'with', 'a', 'white', 'mane.']])
    target.append([['A', 'man', 'strokes', 'a', 'horses', 'nose', 'while', 'it', 'stands', 'over', 'a', 'fence.']])
    return context, target

In [6]:
def preprocess(c, t):
    c_words, t_words, data = [], [], []
    c_words += [word for lists in c for list_ in lists for word in list_]
    t_words += [word for lists in t for list_ in lists for word in list_] 
    for i, lists in enumerate(c):
        for list_ in lists:
            data.append((list_,t[i][0]))
    word_to_idx = {word:i for i, word in enumerate(c_words + t_words)}
    return data, word_to_idx

In [16]:
def train(d, w_idx):
    hidden_size = 128
    dtype = torch.float
    model = TextLSTM()
    optimizer = optim.SGD(model.parameters(), lr = 0.01)
    loss_func = nn.NLLLoss()
    
    for i in range(10):
        total_loss = 0
        for c, t in d:
            context = torch.tensor([w_idx[word] for word in c], dtype = torch.long)
            target = torch.tensor([w_idx[word] for word in t], dtype = torch.long)
            outputs = model(context)
            loss = loss_func(outputs, target)
            optimizer.step()
            loss.backward()
            total_loss += loss
        print("Loss : {}".format(total_loss))
        

In [13]:
def make_batch(seq_data):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word_dict[seq[-1]] # 'e' is target
        print(np.eye(n_class)[input])
        input_batch.append(np.eye(n_class)[input])
        print(target)
        target_batch.append(target)

    return Variable(torch.Tensor(input_batch)), Variable(torch.LongTensor(target_batch))

In [14]:
def main():
    # c, t = data()
    # text_data, word_to_idx = preprocess(c, t)
    seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']
    input_b, target_b = make_batch(seq_data)
    train(input_b, target_b)

In [15]:
if __name__ == "__main__":
    main()

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
4
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
3
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
11
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0

TypeError: __init__() takes 1 positional argument but 4 were given

Raw code

In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

dtype = torch.FloatTensor

char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']
word_dict = {n: i for i, n in enumerate(char_arr)}
number_dict = {i: w for i, w in enumerate(char_arr)}
n_class = len(word_dict) # number of class(=number of vocab)

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

# TextLSTM Parameters
n_step = 3
n_hidden = 128

def make_batch(seq_data):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word_dict[seq[-1]] # 'e' is target
        print(np.eye(n_class)[input])
        input_batch.append(np.eye(n_class)[input])
        print(target)
        target_batch.append(target)

    return Variable(torch.Tensor(input_batch)), Variable(torch.LongTensor(target_batch))

class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()

        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))

    def forward(self, X):
        input = X.transpose(0, 1)  # X : [n_step, batch_size, n_class]

        hidden_state = Variable(torch.zeros(1, len(X), n_hidden))   # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        cell_state = Variable(torch.zeros(1, len(X), n_hidden))     # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  # [batch_size, n_hidden]
        model = torch.mm(outputs, self.W) + self.b  # model : [batch_size, n_class]
        return model

input_batch, target_batch = make_batch(seq_data)

model = TextLSTM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

output = model(input_batch)

# Training
for epoch in range(1000):
    optimizer.zero_grad()

    output = model(input_batch)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

inputs = [sen[:3] for sen in seq_data]

predict = model(input_batch).data.max(1, keepdim=True)[1]
print(inputs, '->', [number_dict[n.item()] for n in predict.squeeze()])



[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
4
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
3
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
11
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0